In [102]:
import numpy as np
import pandas as pd
from pathlib import PurePath
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# def generate_graph_seq2seq_io_data(
#         df, x_offsets, y_offsets, add_time_in_day=True, add_day_in_week=False, scaler=None
# ):
#     """
#     Generate samples from
#     :param df:
#     :param x_offsets:
#     :param y_offsets:
#     :param add_time_in_day:
#     :param add_day_in_week:
#     :param scaler:
#     :return:
#     # x: (epoch_size, input_length, num_nodes, input_dim)
#     # y: (epoch_size, output_length, num_nodes, output_dim)
#     """

#     num_samples, num_nodes = df.shape
#     data = np.expand_dims(df.values, axis=-1)  # data transpose to row to column, reach row value becomes single array
#     feature_list = [data]
#     if add_time_in_day:
#         time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
#         time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
#         feature_list.append(time_in_day)
#     if add_day_in_week:
#         dow = df.index.dayofweek
#         dow_tiled = np.tile(dow, [1, num_nodes, 1]).transpose((2, 1, 0))
#         feature_list.append(dow_tiled)

#     data = np.concatenate(feature_list, axis=-1)
#     x, y = [], []
#     min_t = abs(min(x_offsets))
#     max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
#     for t in range(min_t, max_t):  # t is the index of the last observation.
#         x.append(data[t + x_offsets, ...])
#         y.append(data[t + y_offsets, ...])
#     x = np.stack(x, axis=0)
#     y = np.stack(y, axis=0)
#     return x, y


In [3]:
# def generate_train_val_test(args):
#     seq_length_x, seq_length_y = args.seq_length_x, args.seq_length_y
#     df = pd.read_hdf(args.traffic_df_filename)
#     # 0 is the latest observed sample.
#     x_offsets = np.sort(np.concatenate((np.arange(-(seq_length_x - 1), 1, 1),)))  # random records to split the batches
#     # Predict the next one hour
#     y_offsets = np.sort(np.arange(args.y_start, (seq_length_y + 1), 1))
#     # x: (num_samples, input_length, num_nodes, input_dim)
#     # y: (num_samples, output_length, num_nodes, output_dim)
#     x, y = generate_graph_seq2seq_io_data(
#         df,
#         x_offsets=x_offsets,
#         y_offsets=y_offsets,
#         add_time_in_day=True,
#         add_day_in_week=args.dow,
#     )

#     print("x shape: ", x.shape, ", y shape: ", y.shape)
#     # Write the data into npz file.
#     num_samples = x.shape[0]
#     num_test = round(num_samples * 0.2)
#     num_train = round(num_samples * 0.7)
#     num_val = num_samples - num_test - num_train
#     x_train, y_train = x[:num_train], y[:num_train]
#     x_val, y_val = (
#         x[num_train: num_train + num_val],
#         y[num_train: num_train + num_val],
#     )
#     x_test, y_test = x[-num_test:], y[-num_test:]

#     for cat in ["train", "val", "test"]:
#         _x, _y = locals()["x_" + cat], locals()["y_" + cat]
#         print(cat, "x: ", _x.shape, "y:", _y.shape)
#         np.savez_compressed(
#             os.path.join(args.output_dir, f"{cat}.npz"),
#             x=_x,
#             y=_y,
#             x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
#             y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
#         )

In [4]:
# args = parser.parse_args()
# if os.path.exists(args.output_dir):
#     reply = str(input(f'{args.output_dir} exists. Do you want to overwrite it? (y/n)')).lower().strip()
#     if reply[0] != 'y': exit
# else:
#     os.makedirs(args.output_dir)
# generate_train_val_test(args)


In [134]:
seq_length_x =12
seq_length_y = 12
traffic_df_filename = PurePath('../data/metr-la.h5')
df = pd.read_hdf(traffic_df_filename)
# 0 is the latest observed sample.
x_offsets = np.sort((np.arange(-(seq_length_x - 1), 1, 1)))  # random records to split the batches
# Predict the next one hour
y_start = 1
y_offsets = np.sort(np.arange(y_start, (seq_length_y + 1), 1))

In [135]:
x_offsets

array([-11, -10,  -9,  -8,  -7,  -6,  -5,  -4,  -3,  -2,  -1,   0])

In [7]:
y_offsets

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [8]:
print('x_offsets: ', x_offsets.shape)
print('y_offsets: ', y_offsets.shape)

x_offsets:  (12,)
y_offsets:  (12,)


In [9]:
add_time_in_day=True
add_day_in_week=False

In [10]:
num_samples, num_nodes = df.shape
print('num_sample', num_samples)
print('num_nodes', num_nodes)
print('df-shape', df.shape)

num_sample 34272
num_nodes 207
df-shape (34272, 207)


In [11]:
data = np.expand_dims(df.values, axis=-1)  # data transpose to row to column, reach row value becomes single array
print(data.shape)
print(data[0][:10])

(34272, 207, 1)
[[64.375]
 [67.625]
 [67.125]
 [61.5  ]
 [66.875]
 [68.75 ]
 [65.125]
 [67.125]
 [59.625]
 [62.75 ]]


In [12]:
feature_list = [data]
print(len(feature_list))

1


In [13]:
if add_time_in_day:
  
  # convert/normalize time into readable values by model (daytime(24 + t) - day(24) = time(t))--> t/24
  time_ind = (df.index.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
  
  # creat matrix of date that is similar to the speed matrix as 2nd feature
  time_in_day = np.tile(time_ind, [1, num_nodes, 1]).transpose((2, 1, 0))
  # append speed matrix and time matrix
  feature_list.append(time_in_day)
  print(len(feature_list))
tempdf = pd.DataFrame(time_in_day.reshape(34272, 207))
tempdf.head()


2


,0,1,2,3,4,5,6,7,8,9,...,197,198,199,200,201,202,203,204,205,206
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,...,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472,0.003472
2,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,...,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944,0.006944
3,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,...,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417,0.010417
4,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,...,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889,0.013889


In [15]:
print(feature_list[0].shape)
print(feature_list[1].shape)

(34272, 207, 1)
(34272, 207, 1)


In [14]:
temp_time_ind = (df.index.values[:5] - df.index.values.astype("datetime64[D]")[:5])  / np.timedelta64(1, "D")
temp_time_ind
print(df.index.values[:5])
print(df.index.values.astype("datetime64[D]")[1])
print(df.index.values[:5] - df.index.values.astype("datetime64[D]")[:5])

['2012-03-01T00:00:00.000000000' '2012-03-01T00:05:00.000000000'
 '2012-03-01T00:10:00.000000000' '2012-03-01T00:15:00.000000000'
 '2012-03-01T00:20:00.000000000']
2012-03-01
[            0  300000000000  600000000000  900000000000 1200000000000]


In [60]:
tempt_time_in_day = np.tile(temp_time_ind, [1, 5, 1])#.transpose((2, 1, 0))
print(tempt_time_in_day.shape)
print(tempt_time_in_day)
# (rows, nodes/columns, z_indez)
print(tempt_time_in_day.transpose((2, 1, 0)).shape)
print(tempt_time_in_day.transpose((2, 1, 0)))

(1, 5, 5)
[[[0.         0.00347222 0.00694444 0.01041667 0.01388889]
  [0.         0.00347222 0.00694444 0.01041667 0.01388889]
  [0.         0.00347222 0.00694444 0.01041667 0.01388889]
  [0.         0.00347222 0.00694444 0.01041667 0.01388889]
  [0.         0.00347222 0.00694444 0.01041667 0.01388889]]]
(5, 5, 1)
[[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.00347222]
  [0.00347222]
  [0.00347222]
  [0.00347222]
  [0.00347222]]

 [[0.00694444]
  [0.00694444]
  [0.00694444]
  [0.00694444]
  [0.00694444]]

 [[0.01041667]
  [0.01041667]
  [0.01041667]
  [0.01041667]
  [0.01041667]]

 [[0.01388889]
  [0.01388889]
  [0.01388889]
  [0.01388889]
  [0.01388889]]]


In [ ]:
if add_day_in_week:
  # series with day of the week Mon=0,..., Sun=6
  dow = df.index.dayofweek
  dow_tiled = np.tile(dow, [1, num_nodes, 1]).transpose((2, 1, 0))
  feature_list.append(dow_tiled)

In [29]:
data = np.concatenate(feature_list, axis=-1)


In [20]:
print(data.shape)
print(data[1])

(34272, 207, 2)
[[6.26666667e+01 3.47222222e-03]
 [6.85555556e+01 3.47222222e-03]
 [6.54444444e+01 3.47222222e-03]
 [6.24444444e+01 3.47222222e-03]
 [6.44444444e+01 3.47222222e-03]
 [6.81111111e+01 3.47222222e-03]
 [6.50000000e+01 3.47222222e-03]
 [6.50000000e+01 3.47222222e-03]
 [5.74444444e+01 3.47222222e-03]
 [6.33333333e+01 3.47222222e-03]
 [5.87777778e+01 3.47222222e-03]
 [5.84444444e+01 3.47222222e-03]
 [6.64444444e+01 3.47222222e-03]
 [6.54444444e+01 3.47222222e-03]
 [5.61111111e+01 3.47222222e-03]
 [6.54444444e+01 3.47222222e-03]
 [3.90000000e+01 3.47222222e-03]
 [6.37777778e+01 3.47222222e-03]
 [6.31111111e+01 3.47222222e-03]
 [6.61111111e+01 3.47222222e-03]
 [6.57777778e+01 3.47222222e-03]
 [5.67777778e+01 3.47222222e-03]
 [5.07777778e+01 3.47222222e-03]
 [5.62222222e+01 3.47222222e-03]
 [6.68888889e+01 3.47222222e-03]
 [5.44444444e+01 3.47222222e-03]
 [6.78888889e+01 3.47222222e-03]
 [6.15555556e+01 3.47222222e-03]
 [6.75555556e+01 3.47222222e-03]
 [4.67777778e+01 3.47222222

In [21]:
min_t = abs(min(x_offsets))
max_t = abs(num_samples - abs(max(y_offsets)))  # Exclusive
print('min_t', min_t)
print('max_t', max_t)

min_t 11
max_t 34260


In [64]:
x, y = [], []
for t in range(min_t, max_t):  # t is the index of the last observation.
  x.append(data[t + x_offsets, ...])
  y.append(data[t + y_offsets, ...])
x = np.stack(x, axis=0)
y = np.stack(y, axis=0)

In [79]:
temp_x, temp_y = [], []
temp_data = np.arange(0, 100, 1)
temp_x_offsets = np.array([ -1, 0])
temp_y_offsets = np.array([1, 2])
print(temp_data.shape)
for t in range(2, 98):  # t is the index of the last observation.
  temp_x.append(temp_data[t + temp_x_offsets, ...])
  temp_y.append(temp_data[t + temp_y_offsets, ...])

(100,)


In [82]:
print(temp_x)
print('------')
print(temp_y)
print('------')

[array([1, 2]), array([2, 3]), array([3, 4]), array([4, 5]), array([5, 6]), array([6, 7]), array([7, 8]), array([8, 9]), array([ 9, 10]), array([10, 11]), array([11, 12]), array([12, 13]), array([13, 14]), array([14, 15]), array([15, 16]), array([16, 17]), array([17, 18]), array([18, 19]), array([19, 20]), array([20, 21]), array([21, 22]), array([22, 23]), array([23, 24]), array([24, 25]), array([25, 26]), array([26, 27]), array([27, 28]), array([28, 29]), array([29, 30]), array([30, 31]), array([31, 32]), array([32, 33]), array([33, 34]), array([34, 35]), array([35, 36]), array([36, 37]), array([37, 38]), array([38, 39]), array([39, 40]), array([40, 41]), array([41, 42]), array([42, 43]), array([43, 44]), array([44, 45]), array([45, 46]), array([46, 47]), array([47, 48]), array([48, 49]), array([49, 50]), array([50, 51]), array([51, 52]), array([52, 53]), array([53, 54]), array([54, 55]), array([55, 56]), array([56, 57]), array([57, 58]), array([58, 59]), array([59, 60]), array([60, 6

In [83]:
temp_x = np.stack(temp_x, axis=0)
temp_y = np.stack(temp_y, axis=0)

In [87]:
print(temp_x[:10], )
print('------')
print(temp_y[:10])
print('------')

[[ 1  2]
 [ 2  3]
 [ 3  4]
 [ 4  5]
 [ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 9 10]
 [10 11]]
------
[[ 3  4]
 [ 4  5]
 [ 5  6]
 [ 6  7]
 [ 7  8]
 [ 8  9]
 [ 9 10]
 [10 11]
 [11 12]
 [12 13]]
------


In [93]:
print(len(x))
print(len(y))
print(x[1].shape)

34249
34249
(12, 207, 2)


In [94]:
print("x shape: ", x.shape, ", y shape: ", y.shape)


x shape:  (34249, 12, 207, 2) , y shape:  (34249, 12, 207, 2)


In [95]:
# Write the data into npz file.
num_samples = x.shape[0]
print(num_samples)
num_test = round(num_samples * 0.2)
print(num_test)
num_train = round(num_samples * 0.7)
print(num_train)
num_val = num_samples - num_test - num_train
print(num_val)

34249
6850
23974
3425


In [96]:
x_train, y_train = x[:num_train], y[:num_train]
x_val, y_val = (
    x[num_train: num_train + num_val],
    y[num_train: num_train + num_val],
)
x_test, y_test = x[-num_test:], y[-num_test:]

In [123]:
# path = os.path.sys.path[0]+'/data_gen/'
# for cat in ["train", "val", "test"]:
#     _x, _y = locals()["x_" + cat], locals()["y_" + cat]
#     print(cat, "x: ", _x.shape, "y:", _y.shape)
#     np.savez_compressed(
#       path + f"{cat}.npz",
#       x=_x,
#       y=_y,
#       x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
#       y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
#     )

In [124]:
train_x = np.load(PurePath('../data/METR-LA/train.npz'))

In [130]:
train_x['x'].shape

(23974, 12, 207, 2)

In [141]:
(64 - (len(train_x['x']) % 64)) % 64

26

In [136]:
len(train_x['y'])#.shape

23974

In [132]:
train_x['x_offsets']

array([[-11],
       [-10],
       [ -9],
       [ -8],
       [ -7],
       [ -6],
       [ -5],
       [ -4],
       [ -3],
       [ -2],
       [ -1],
       [  0]])

In [133]:
train_x['y_offsets']

array([[ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12]])

In [125]:
df = pd.DataFrame(train_x)

In [126]:
df.head()

,0
0,x
1,y
2,x_offsets
3,y_offsets
